In [ ]:
import pandas as pd
import numpy as np
import gspread
from google.oauth2.service_account import Credentials
import joblib

# ✅ Google Sheets API Credentials
SERVICE_ACCOUNT_FILE = "your_service_account.json"
SCOPES = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# ✅ Load Google Sheet
SHEET_ID = "YOUR_GOOGLE_SHEET_ID"
SHEET_NAME = "Sheet1"
sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)

# ✅ Load Trained SOH Model
model = joblib.load("rf_soh_model.pkl")

def fetch_google_sheet_data():
    """ Fetches the latest 15-minute battery data from Google Sheets. """
    data = sheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])  # Convert to DataFrame
    df = df.astype({"Voltage": float, "Current": float, "Capacity": float, "Hour": int, "Minute": int, "Seconds": int})
    
    # ✅ Extract last 15 minutes of data
    df['Time (s)'] = df['Hour'] * 3600 + df['Minute'] * 60 + df['Seconds']
    df = df[df['Time (s)'] >= (df['Time (s)'].max() - 899)]  # Keep last 15 min
    
    return df

def estimate_soh():
    """ Estimates SOH using the trained model. """
    df = fetch_google_sheet_data()

    if df.empty:
        print("🚨 No data available for SOH estimation.")
        return

    # ✅ Prepare Data for Model
    features = df[['Voltage', 'Current']]
    soh_pred = model.predict(features)

    # ✅ Calculate Mean SOH
    estimated_soh = np.mean(soh_pred)

    # ✅ Write Back to Google Sheets
    sheet.update_cell(2, 8, f"{estimated_soh:.2f}%")  # Update SOH in 2nd row, Column H
    
    print(f"✅ SOH Estimation Completed: {estimated_soh:.2f}%")

if __name__ == "__main__":
    estimate_soh()
